In [1]:
import torch 
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim

In [37]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)
class model(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, out_channel)
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 1600)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = model(1, 10)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(model, trainloader, optimizer, criterion, epochs=5):
    model.to('cuda')
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for images, labels in trainloader:
            images, labels = images.to('cuda'), labels.to('cuda')
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Training loss: {running_loss/len(trainloader)}")
#train(model, trainloader, optimizer, criterion)


In [38]:
train(model, trainloader, optimizer, criterion)

Training loss: 0.10755692857802533
Training loss: 0.04230285436622516
Training loss: 0.028938689454678306
Training loss: 0.02179226435291238
Training loss: 0.017482458466464217


In [44]:
def eval(model):
    validset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
    validloader = torch.utils.data.DataLoader(validset, batch_size=8, shuffle=True)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to('cuda'), labels.to('cuda')
            output = model(images)
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

In [41]:
class model_nn(nn.Module):
    def __init__(self):
        super(model_nn, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [42]:
model_nn = model_nn()
optimizer = optim.Adam(model_nn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train(model_nn, trainloader, optimizer, criterion)

Training loss: 0.30273927864553407
Training loss: 0.16297960298563316
Training loss: 0.13270295180001074
Training loss: 0.11461276874116277
Training loss: 0.10422792640142318


In [45]:
eval(model_nn)

Accuracy: 0.9687
